<a href="https://colab.research.google.com/github/RohinSequeira/EVA6_Session3_Pytorch/blob/main/EVA6_session3_pytroch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import print_function #Allowing the interpreter to use old version of python with features of new verion, in this case, print
import torch # importing torch library
import torch.nn as nn #torch.nn provides basic building blocks for graphs, as per pytorch documentation
import torch.nn.functional as F # importing all the functions in torch.nn library
import torch.optim as optim # contains various optimization algorithms
from torchvision import datasets, transforms #contains popular datasets and commom image transformations for computer vision
from torchsummary import summary #provides clean and clear visual summary of the model
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import TensorDataset
from torch import Tensor

In [2]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

In [3]:
class Net(nn.Module):
    def __init__(self):
      """
      In this constuctor, we define parameters, representing the different layers that will make
      up the Neural Network
      """
      super(Net, self).__init__()

      self.conv1 = nn.Conv2d(1, 32, 3, padding=1) #input -? OUtput? RF
      self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
      self.pool1 = nn.MaxPool2d(2, 2)
      self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
      self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
      self.pool2 = nn.MaxPool2d(2, 2)
      self.conv5 = nn.Conv2d(256, 512, 3)
      self.conv6 = nn.Conv2d(512, 1024, 3)
      self.conv7 = nn.Conv2d(1024, 10, 3)

      # First fully connected layer
      self.fc1 = nn.Linear(11, 5, bias=False)
      self.fc2 = nn.Linear(5, 1, bias=False)

    def forward(self, x1, x2):
      """
      To compute output Tensors from the input Tensors. Sequence of layers are defined here.
      """
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x1)))))
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
        x = F.relu(self.conv6(F.relu(self.conv5(x))))
        x = self.conv7(x)
        x = x.view(-1, 10)

        n1 = torch.argmax(x,dim=1).reshape(-1,1)
        n2 = torch.cat((n1,x2),dim=1)
        n  = self.fc2(self.fc1(n2))

        return F.log_softmax(x), n

In [4]:
my_nn = Net().to(device)

In [5]:
# Download and load MNIST datasets for train and test of Convolutional Network. 
# torch.utils.data.DataLoader class provided by pytorch is used for this purpose.
torch.manual_seed(1)
batch_size = 64

train_data = datasets.MNIST('../data', train=True, download=True, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ]))

test_data = datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ]))
train_random_input = torch.randint(0,10,(60000,))
train_target2 = torch.add(train_random_input, train_data.targets)

test_random_input = torch.randint(0,10,(10000,))
test_target2 = torch.add(test_random_input, test_data.targets)
train_one_hot = torch.nn.functional.one_hot(train_random_input)
test_one_hot = torch.nn.functional.one_hot(test_random_input)

kwargs = {'num_workers': 2, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(train_data,
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(test_data,
    batch_size=batch_size, shuffle=False, **kwargs)

from torch.utils.data import TensorDataset

random_train_dataset = TensorDataset(train_one_hot, train_target2)
random_train_loader = torch.utils.data.DataLoader(random_train_dataset, batch_size=batch_size, shuffle=False, **kwargs)

random_test_dataset = TensorDataset(test_one_hot, test_target2)
random_test_loader = torch.utils.data.DataLoader(random_test_dataset, batch_size=batch_size, shuffle=False, **kwargs)

In [6]:
learning_rate = 0.001

optimizer = torch.optim.SGD(my_nn.parameters(), lr=learning_rate) 

In [7]:
def train(model, device, train_loader1, train_loader2, optimizer, epoch):
  """
  Define Train functionality.
  """
    model.train()
    for batch_idx, (dt_1, dt_2) in enumerate(zip(train_loader1, train_loader2)):
        data1, data2, target1,  target2 = dt_1[0].to(device), dt_2[0].float().to(device), dt_1[1].to(device), dt_2[1].type(torch.LongTensor).to(device)
        optimizer.zero_grad()
        output1, output2 = model(data1, data2)
        criterion1 = nn.NLLLoss()
        loss1 = criterion1(output1, target1)
        criterion2 = nn.MSELoss()
        loss2 = criterion2(output2, target2.float())
        loss = loss1 + loss2
        loss.backward()
        optimizer.step()
        if batch_idx % 50 == 0:
          print(f'loss of image {loss1}')
          print(f'loss of summed value {loss2}')
          print(f'loss={loss.item()} batch_id={batch_idx}')
    return 0

In [8]:
#Trigger Model Train and Test in epochs with for loop.
optimizer.zero_grad()
for epoch in range(1, 6):
    # Train the model using all the defined objects
    predicted = train(my_nn, device, train_loader, random_train_loader, optimizer, epoch)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


loss of image 2.302597761154175
loss of summed value 93.62191009521484
loss=95.92450714111328 batch_id=0
loss of image 2.3043293952941895
loss of summed value 31.53976821899414
loss=33.84409713745117 batch_id=50
loss of image 2.3004634380340576
loss of summed value 14.67499828338623
loss=16.975461959838867 batch_id=100
loss of image 2.297588348388672
loss of summed value 15.166685104370117
loss=17.46427345275879 batch_id=150
loss of image 2.297001361846924
loss of summed value 15.455636978149414
loss=17.75263786315918 batch_id=200
loss of image 2.297227621078491
loss of summed value 14.183305740356445
loss=16.480533599853516 batch_id=250
loss of image 2.2977945804595947
loss of summed value 19.940078735351562
loss=22.237873077392578 batch_id=300
loss of image 2.2974205017089844
loss of summed value 13.721067428588867
loss=16.01848793029785 batch_id=350
loss of image 2.296058177947998
loss of summed value 20.344310760498047
loss=22.640369415283203 batch_id=400
loss of image 2.2962274551

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


loss of image 2.279109239578247
loss of summed value 41.14697265625
loss=43.426082611083984 batch_id=0
loss of image 2.272397994995117
loss of summed value 40.81637191772461
loss=43.088768005371094 batch_id=50
loss of image 2.270951509475708
loss of summed value 33.34664535522461
loss=35.61759567260742 batch_id=100
loss of image 2.2684812545776367
loss of summed value 40.48430633544922
loss=42.75278854370117 batch_id=150
loss of image 2.26603627204895
loss of summed value 35.03494644165039
loss=37.30098342895508 batch_id=200
loss of image 2.2586801052093506
loss of summed value 35.84098815917969
loss=38.099666595458984 batch_id=250
loss of image 2.2561652660369873
loss of summed value 36.338592529296875
loss=38.594757080078125 batch_id=300
loss of image 2.2557241916656494
loss of summed value 25.797340393066406
loss=28.053064346313477 batch_id=350
loss of image 2.2309494018554688
loss of summed value 23.618003845214844
loss=25.848953247070312 batch_id=400
loss of image 2.21672844886779

In [12]:
def test(model, device, test_loader1, test_loader2):
  """
  # Define Test functionality, along with measuring accuracy of prediction.
  """
    model.eval()
    test_loss = 0
    correct = 0
    predicted_sum=torch.FloatTensor().to(device)
    for batch_idx, (dt_1, dt_2) in enumerate(zip(test_loader1, test_loader2)):
        data1, data2,target1, target2 = dt_1[0].to(device), dt_2[0].float().to(device), dt_1[1].to(device), dt_2[1].to(device)
        output1, output2 = model(data1, data2)
        criterion1 = nn.NLLLoss() #Loss function for MNIST
        loss1 = criterion1(output1, target1)
        criterion2 = nn.MSELoss() #Loss function for sum of digits
        loss2 = criterion2(output2, target2.float()) #Cumulative loss
        test_loss += float(loss1 + loss2)
        pred = output1.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        correct += pred.eq(target1.view_as(pred)).sum().item()
        predicted_sum=torch.cat((predicted_sum,output2.detach().reshape(-1)))
        
    
    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    return predicted_sum

In [10]:
del train_data, train_random_input, train_target2

In [13]:
pred = test(my_nn, device, test_loader, random_test_loader)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)



Test set: Average loss: 0.2655, Accuracy: 9180/10000 (92%)



/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([16])) that is different to the input size (torch.Size([16, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [22]:
#Measure R2 score as metric for final data sum. Output sucks, we know.
from sklearn.metrics import r2_score
r2_score(test_target2, pred.cpu())


0.06546127184631612